In [ ]:
#https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

# 1. Import Dependencies

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Types of Spaces

In [22]:
Discrete(2)

0

In [105]:
Box(low=np.array([-90]), high=np.array([90])).sample()
# np.array([0]).astype(float)
np.array([0], dtype=np.float32)[-1]


0.0

In [ ]:
Box(0,255,shape=(3,3), dtype=int).sample()

In [ ]:
Tuple((Discrete(2), Box(0,100, shape=(1,)))).sample()

In [25]:
Dict({'height':Discrete(2), "speed":Box(0,100, shape=(1,))}).sample()

OrderedDict([('height', 0), ('speed', array([80.04878], dtype=float32))])

In [23]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [24]:
MultiDiscrete([5,2,2]).sample()

array([1, 1, 1])

# 3. Building an Environment

In [120]:
class SelfBalance(Env):
    def __init__(self):
        # Actions we can take: Forward, Backward
        self.action_space = Discrete(3)
        # Angle of Robot
        self.observation_space = Box(low=np.array([-90]), high=np.array([90]))
        # Set start state
        self.state = 0
        # Set episode length
        self.episode_length = 60
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        self.state += (action -1)*10 
        # Reduce episode length by 1 second
        self.episode_length -= 1 
        
        # Calculate reward
        if self.state >=-15 and self.state <=15: 
            reward = np.abs(self.state)
        else: 
            reward = -1 * (0.167 * (np.abs(self.state) - 15))
        
        # Check if episode is done
        if self.episode_length <= 0: 
            done = True
        else:
            done = False
        
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset robot
        self.state = np.array([0], dtype=np.float32)
        # Reset episode time
        self.episode_length = 60 
        return self.state

In [121]:
env=SelfBalance()

/home/rigved/anaconda3/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [49]:
env.observation_space.sample()

array([59.3312], dtype=float32)

In [112]:
env.reset()

array([0.], dtype=float32)

In [38]:
from stable_baselines3.common.env_checker import check_env

In [122]:
check_env(env, warn=True)

AssertionError: The observation returned by `reset()` method must be a numpy array

# 4. Test Environment

In [98]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-2280.0
Episode:2 Score:-2060.0
Episode:3 Score:-830.0
Episode:4 Score:-2890.0
Episode:5 Score:-3170.0


In [ ]:
env.close()

# 5. Train Model

In [80]:
log_path = os.path.join('Training', 'Logs')

In [82]:
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [85]:
model.learn(total_timesteps=40000)

Logging to Training/Logs/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -52.3    |
| time/              |          |
|    fps             | 2216     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | -52.5        |
| time/                   |              |
|    fps                  | 1661         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0023442474 |
|    clip_fraction        | 0.00269      |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.688       |
|    explained_variance   |

# 6. Save Model

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)